# 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 계층(layer)모듈(module)로구 구성되어 있습니다. `torch.nn` 네임스페이스는 신경망을 구성하는데 필요한 모든 구성요소를 제공합니다. Pytorch의 모든 모듈은 `nn.Module`의 하위 클래스(subclass)입니다. 신경망은 다른 모듈 (계층: layer)로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms

가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.cuda.is_available()
    else "cpu"
)
print("Using {} device".format(device))

Using cpu device


## 클래스 정의하기

신경망 모델을 `nn.Moduule`의 하위클래스를 정의하고 `__init__`에서 신경망 계층들을 초기화 합니다. `nn.Module`을 상속받은 모든 클래스는 `forward` 메소드에 입력 데이터에 대한 연산들을 구현합니다. `forward` 함수는 출력을 계산하기 위해 사용되는 입력 텐서를 인자로 받고 출력 텐서를 반환합니다. 신경망의 학습 가능한 매개변수들은 `net.parameters()`에 의해 반환됩니다.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 *백그래운드 연산들* 과 함께 모델의 `forward`를 실행합니다.

모델에 입력을 전달하여 호출하면 2차원 텐서를 반한합니다. 2차원 텐서의 dim=0은 각 분류(class)에 대한 원시(raw)예측값 10개가, dim=1에는 각 출력의 개별 값들이 해당합니다. 원시 예측값을 `nn.Softmax` 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


## 모델 계층(Layer)

FashionMNIST 모델의 계층들을 살펴보겠습니다. 이를 설명하기위해 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 살펴보겠습니다.

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

*nn.Flatten* 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀의 연속된 배열로 변환합니다. 이 계층은 일부 모델의 첫번째 계층으로 사용될 수 있습니다. 이 계층은 배치 차원은 유지하고 나머지 차원을 평탄화(flatten)합니다.

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

*선형 계층*은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환을 적용합니다. 이 계층은 `y = x*W^T + b`와 같은 선형 변환을 표현합니다. 선형 계층은 신경망에 의해 학습되는 가중치와 편향이 있습니다.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU

비선형 활성화는 선형 변환 후에 신경망에 비선형성을 도입하는 것을 의미합니다. 비선형 활성화는 신경망이 다양한 현상을 학습할 수 있도록 도와줍니다.

비선형 활성화로는 `nn.ReLU` 모듈을 사용하겠습니다. 이 모듈은 입력 요소 중 음수를 0으로 만드는 [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) 함수를 적용합니다.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4822, -0.0868, -0.7402,  0.1496, -0.6520,  0.2991, -0.1463, -0.2545,
          0.1787, -0.2979, -0.2811,  0.6926, -0.5475, -0.2998, -0.1615, -0.1991,
          0.3124, -0.1241,  0.5573, -0.1376],
        [ 0.0105, -0.2967, -0.6184, -0.2206, -0.4630, -0.2625, -0.1177, -0.3010,
          0.4181, -0.5878, -0.3041,  0.5424, -0.2098,  0.2949, -0.4042, -0.1719,
          0.4904, -0.2251,  0.2855,  0.1294],
        [-0.2868,  0.1456, -0.8435, -0.2082, -0.5791,  0.1722, -0.2587, -0.5667,
          0.4279, -0.1445, -0.4307,  0.5665, -0.2199, -0.0317,  0.0182, -0.0574,
          0.2739, -0.2067,  0.1259, -0.0288]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.1496, 0.0000, 0.2991, 0.0000, 0.0000, 0.1787,
         0.0000, 0.0000, 0.6926, 0.0000, 0.0000, 0.0000, 0.0000, 0.3124, 0.0000,
         0.5573, 0.0000],
        [0.0105, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4181,
         0.0000, 0.0000, 0.5424, 0.0000, 0.2949, 0.00

## nn.Sequential

*nn.Sequential*은 순서를 갖는 모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈을 통해 전달됩니다. 이 컨테이너는 신경망의 순차적인 계층을 구성하는데 사용됩니다. FashionMNIST 신경망을 `nn.Sequential`을 사용하여 정의해보겠습니다.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

## nn.Softmax

신경망의 마지막 선형 계층은 *nn.Softmax* 모듈에 통과시킵니다. 이 모듈은 입력값을 `[0, 1]` 범위 안의 값으로 정규화하며 출력값의 총합은 1입니다. 이 출력은 각 클래스에 대한 모델의 예측 확률입니다.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 모델 매개변수

신경망 내부의 많은 계층들은 매개변수화됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. `nn.Module`을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, `parameters()` 및 `named_parameters()` 메소드로 모든 매개변수에 접근할 수 있습니다.

이 예제에서는 각 매개변수들을 반복(iterate)하고 값을 출력하고, 최적화할 변수들의 갱신(update)을 위해 이터레이션(iteration)마다 변화도(gradient)를 0으로 설정합니다.

In [12]:
print("Model structure: ", model, "\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0029, -0.0090,  0.0344,  ...,  0.0266,  0.0147,  0.0312],
        [-0.0159, -0.0253,  0.0163,  ..., -0.0314, -0.0058,  0.0140]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0124, -0.0355], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0438,  0.0084,  0.0347,  ...,  0.0375, -0.0126,  0.0384],
        [ 0.0091,  0.0177,  0.0191,  ...,  0.0177, -0.0344,  0.0116]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_